In [15]:
import numpy as np
import pandas as pd
import math
from scipy import stats

## Question 1: Benjamini-Hochberg Procedure

In [3]:
#construct data
n = 20
m = 1000

np.random.seed (5)
re = np.random.normal(size=(n,m))
#print(re)

In [4]:
#conduct t-test

t = stats.ttest_1samp(re,popmean=0,axis=0)

In [5]:
#Control type I error for each null hypothesis at level alpha = 0.05, count how many null hypotheses are rejected
print(sum(i < 0.05 for i in t.pvalue))

55


In [6]:
#Control the FDR at level 0.2 using Benjamini-Hochberg Procedure,count how many null hypotheses are rejected

sort_p = np.sort(t.pvalue)

count = 0
for i in range(1000):
    if sort_p[i] < 0.2*(i+1)/1000:
        print(sort_p[i],"<",0.2*(i+1)/1000)
        count+=1
    else:
        print(sort_p[i],">=",0.2*(i+1)/1000)

0.00011063785520497688 < 0.0002
0.00017449247205107008 < 0.0004
0.001357507902708003 >= 0.0006000000000000001
0.0015232104366556419 >= 0.0008
0.0032450627217733073 >= 0.001
0.00338225118350119 >= 0.0012000000000000001
0.004929280372293247 >= 0.0014000000000000002
0.006983400101333055 >= 0.0016
0.007182201894412108 >= 0.0018
0.009596231968802039 >= 0.002
0.009974769242155783 >= 0.0022
0.010505388873366528 >= 0.0024000000000000002
0.011003537194694168 >= 0.0026
0.011013123232311612 >= 0.0028000000000000004
0.011438831410577386 >= 0.003
0.013301718424467736 >= 0.0032
0.013833241535196264 >= 0.0034000000000000002
0.014800892585144606 >= 0.0036
0.016415879051500534 >= 0.0038000000000000004
0.0167223944068755 >= 0.004
0.01706722143371989 >= 0.004200000000000001
0.017237875421698644 >= 0.0044
0.020492919576432806 >= 0.004600000000000001
0.021242151356943136 >= 0.0048000000000000004
0.022210933733300815 >= 0.005
0.023312537607020248 >= 0.0052
0.02555910756063533 >= 0.0054
0.026092533343146214 

In [7]:
print(count)

2


## Question 2: Tree

In [39]:
arr = np.array([[0.33, -0.53], 
              [0.32, -1.07],
              [0.88, 2.16],
              [0.01, -1.25],
              [0.71, -2.32]])
df = pd.DataFrame(arr, columns = ['X','Y'])

In [40]:
df = df.sort_values('X')
cutpoint = np.zeros(5)
for i in range(len(cutpoint)):
    cutpoint[i] = df.iloc[i,0]

In [41]:
cutpoint

array([0.01, 0.32, 0.33, 0.71, 0.88])

In [42]:
def get_XL(cut, data):
    left_list = list()
    for i in data.iloc[:,0]:
        if i < cut:
            left_list.append(i)
        else:
            pass
    return left_list

def get_XR(cut, data):
    right_list = list()
    for i in data.iloc[:,0]:
        if i >= cut:
            right_list.append(i)
        else:
            pass
    return right_list

def get_YL(cut, data):
    left_list = list()
    for i in range(len(data)):
        if data.iloc[i,0] < cut:
            left_list.append(data.iloc[i,1])
        else:
            pass
    return left_list

def get_YR(cut, data):
    right_list = list()
    for i in range(len(data)):
        if data.iloc[i,0] >= cut:
            right_list.append(data.iloc[i,1])
        else:
            pass
    return right_list


In [43]:
def mse_calculate(lst):
    x = 0
    for i in lst:
        x += (i - sum(lst)/len(lst))**2 / len(lst)
    return x

In [44]:
##depth=0
MSE_list = []
MSE = 0

for k in range(len(df['Y'])):
    MSE += (df.iloc[k,1] - (sum(df['Y'])/len(df['Y'])))**2
result = 'root node:',(df['Y'].values)
print('depth=0:',result)
MSE_list.append(MSE) ##weighted mean square error
best_split =result


for j in cutpoint:
    ##depth = 1
    X_L = get_XL(j, df)
    X_R = get_XR(j, df)
    Y_L = get_YL(j, df)
    Y_R = get_YR(j, df)

    mse_left = mse_calculate(Y_L)
    mse_right = mse_calculate(Y_R)

    MSE = (mse_left*len(Y_L) + mse_right*len(Y_R)) / 5
    result = 'left :',Y_L ,'right:',Y_R
    print('depth=1:',result)
    if MSE < min(MSE_list):
        best_split = result
    MSE_list.append(MSE)

print('best split:',best_split)

# any number in (0.71, 0.88] is ok
the_cut = 0.81
X_L, X_R, Y_L, Y_R = get_XL(the_cut, df), get_XR(the_cut, df), get_YL(the_cut, df),get_YR(the_cut, df)
for m in X_L:
    for n in X_R:
        if m >= n:
            pass
        else:
            df_L = pd.DataFrame({'X':X_L, 'Y':Y_L})
            X_LL, X_LR, Y_LL, Y_LR = get_XL(m, df_L), get_XR(m, df_L), get_YL(m, df_L), get_YR(m, df_L)
            
            df_R = pd.DataFrame({'X':X_R, 'Y':Y_R})
            X_RL, X_RR, Y_RL, Y_RR = get_XL(n, df_R), get_XR(n, df_R), get_YL(n, df_R), get_YR(n, df_R)

            mse_LL = mse_calculate(Y_LL)
            mse_LR = mse_calculate(Y_LR)
            mse_RL = mse_calculate(Y_RL)
            mse_RR = mse_calculate(Y_RR)

            MSE_L = (mse_LL*len(Y_LL) + mse_LR*len(Y_LR)) / 5
            MSE_R = (mse_RL*len(Y_RL) + mse_RR*len(Y_RR)) / 5
            MSE = MSE_L + MSE_R
            result = 'LL :',Y_LL ,'LR:',Y_LR, 'RL :',Y_RL ,'RR:',Y_RR
            print('一階左右都有:',result)
            if MSE < min(MSE_list):
                best_split = result
            MSE_list.append(MSE)
print('best split:',best_split)

depth=0: ('root node:', array([-1.25, -1.07, -0.53, -2.32,  2.16]))
depth=1: ('left :', [], 'right:', [-1.25, -1.07, -0.53, -2.32, 2.16])
depth=1: ('left :', [-1.25], 'right:', [-1.07, -0.53, -2.32, 2.16])
depth=1: ('left :', [-1.25, -1.07], 'right:', [-0.53, -2.32, 2.16])
depth=1: ('left :', [-1.25, -1.07, -0.53], 'right:', [-2.32, 2.16])
depth=1: ('left :', [-1.25, -1.07, -0.53, -2.32], 'right:', [2.16])
best split: ('left :', [-1.25, -1.07, -0.53, -2.32], 'right:', [2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25, -1.07, -0.53, -2.32], 'RL :', [], 'RR:', [2.16])
一階左右都有: ('LL :', [-1.25], 'LR:', [-1.07, -0.53, -2.32], 'RL :', [], 'RR:', [2.16])
一階左右都有: ('LL :', [-1.25, -1.07], 'LR:', [-0.53, -2.32], 'RL :', [], 'RR:', [2.16])
一階左右都有: ('LL :', [-1.25, -1.07, -0.53], 'LR:', [-2.32], 'RL :', [], 'RR:', [2.16])
best split: ('LL :', [-1.25, -1.07, -0.53], 'LR:', [-2.32], 'RL :', [], 'RR:', [2.16])


In [45]:
avg = (-1.25-1.07-0.53)/3
avg

-0.9500000000000002

In [46]:
MSE_list = [111]
gini = 0

In [47]:
##depth = 2
for j in cutpoint:
    ##depth = 1
    X_L, X_R, Y_L, Y_R = get_XL(j, df), get_XR(j, df), get_YL(j, df),get_YR(j, df)

    # 對第二層左list 算第二層的左右list的mse
    if len(X_R) >= 1 and len(X_L) == 0:
        for m in X_R:
            df_R = pd.DataFrame({'X':X_R, 'Y':Y_R})
            X_RL, X_RR, Y_RL, Y_RR = get_XL(m, df_R), get_XR(m, df_R), get_YL(m, df_R), get_YR(m, df_R)

            mse_left2 = mse_calculate(Y_RL)
            mse_right2 = mse_calculate(Y_RR)
            MSE = (mse_left2*len(Y_RL) + mse_right2*len(Y_RR)) / 5
            result = "LL:",[], "LR:",[],  'RL :',Y_RL ,'RR',Y_RR
            print('一階左空:',result)
            if MSE < min(MSE_list):
                best_split = result
            MSE_list.append(MSE)

    elif len(X_L) >= 1 and len(X_R) >= 1:
        for m in X_L:
            for n in X_R:
                if m >= n:
                    pass
                else:
                    df_L = pd.DataFrame({'X':X_L, 'Y':Y_L})
                    X_LL, X_LR, Y_LL, Y_LR = get_XL(m, df_L), get_XR(m, df_L), get_YL(m, df_L), get_YR(m, df_L)
                    
                    df_R = pd.DataFrame({'X':X_R, 'Y':Y_R})
                    X_RL, X_RR, Y_RL, Y_RR = get_XL(n, df_R), get_XR(n, df_R), get_YL(n, df_R), get_YR(n, df_R)

                    mse_LL = mse_calculate(Y_LL)
                    mse_LR = mse_calculate(Y_LR)
                    mse_RL = mse_calculate(Y_RL)
                    mse_RR = mse_calculate(Y_RR)

                    MSE_L = (mse_LL*len(Y_LL) + mse_LR*len(Y_LR)) / 5
                    MSE_R = (mse_RL*len(Y_RL) + mse_RR*len(Y_RR)) / 5
                    MSE = MSE_L + MSE_R
                    result = 'LL :',Y_LL ,'LR:',Y_LR, 'RL :',Y_RL ,'RR:',Y_RR
                    print('一階左右都有:',result)
                    if MSE < min(MSE_list):
                        best_split = result
                    MSE_list.append(MSE)
print(MSE)
print('best split:',best_split)

一階左空: ('LL:', [], 'LR:', [], 'RL :', [], 'RR', [-1.25, -1.07, -0.53, -2.32, 2.16])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [-1.25], 'RR', [-1.07, -0.53, -2.32, 2.16])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [-1.25, -1.07], 'RR', [-0.53, -2.32, 2.16])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [-1.25, -1.07, -0.53], 'RR', [-2.32, 2.16])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [-1.25, -1.07, -0.53, -2.32], 'RR', [2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25], 'RL :', [], 'RR:', [-1.07, -0.53, -2.32, 2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25], 'RL :', [-1.07], 'RR:', [-0.53, -2.32, 2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25], 'RL :', [-1.07, -0.53], 'RR:', [-2.32, 2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25], 'RL :', [-1.07, -0.53, -2.32], 'RR:', [2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25, -1.07], 'RL :', [], 'RR:', [-0.53, -2.32, 2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25, -1.07], 'RL :', [-0.53], 'RR:', [-2.32, 2.16])
一階左右都有: ('LL :', [], 'LR:', [-1.25, -1.07], 'RL :', [-0.53, -2.32], 'RR:', [2.16])
一階左右都有: 

# Q2 (e)(f)

let dog = 0, cat=1

In [48]:
arr = np.array([[0.33, 1], 
              [0.32, 1],
              [0.88, 0],
              [0.01, 0],
              [0.71, 1]])
df = pd.DataFrame(arr, columns = ['X','Y'])

In [49]:
df = df.sort_values('X')

In [50]:
def gini_calculate(lst):
    if len(lst)==0:
        return 0
    else:
        x = (lst.count(0)*lst.count(1))/len(lst)
        return x

In [52]:
##depth = 2
gini_list = []
gini = 0
result_list = []

for j in cutpoint:
    ##depth = 1
    X_L, X_R, Y_L, Y_R = get_XL(j, df), get_XR(j, df), get_YL(j, df),get_YR(j, df)
    # 對第二層左list 算第二層的左右list的mse
    if len(X_R) >= 1 and len(X_L) == 0:
        for m in X_R:
            giniRL, giniRR = 0,0
            df_R = pd.DataFrame({'X':X_R, 'Y':Y_R})
            X_RL, X_RR, Y_RL, Y_RR = get_XL(m, df_R), get_XR(m, df_R), get_YL(m, df_R), get_YR(m, df_R)
            if len(Y_RL)>0:
                giniRL = (Y_RL.count(1) * Y_RL.count(0)) /len(Y_RL) 
            if len(Y_RR)>0:
                giniRR = (Y_RR.count(1) * Y_RR.count(0)) /len(Y_RR)
            gini = giniRL + giniRR
            result = "LL:",[], "LR:",[],  'RL :',Y_RL ,'RR',Y_RR
            result_list.append(result)
            print('一階左空:',result)
            gini_list.append(gini)

    elif len(X_L) >= 1 and len(X_R) >= 1:
        for m in X_L:
            for n in X_R:
                if m >= n:
                    pass
                else:
                    giniRL, giniRR, giniLL, giniLR = 0,0,0,0
                    df_L = pd.DataFrame({'X':X_L, 'Y':Y_L})
                    X_LL, X_LR, Y_LL, Y_LR = get_XL(m, df_L), get_XR(m, df_L), get_YL(m, df_L), get_YR(m, df_L)
                    
                    df_R = pd.DataFrame({'X':X_R, 'Y':Y_R})
                    X_RL, X_RR, Y_RL, Y_RR = get_XL(n, df_R), get_XR(n, df_R), get_YL(n, df_R), get_YR(n, df_R)
                    if len(Y_RL)>0:
                        giniRL = (Y_RL.count(1) * Y_RL.count(0)) /len(Y_RL) 
                    if len(Y_RR)>0:
                        giniRR = (Y_RR.count(1) * Y_RR.count(0)) /len(Y_RR)
                    if len(Y_LL)>0:
                        giniLL = (Y_LL.count(1) * Y_LL.count(0)) /len(Y_LL)
                    if len(Y_LR)>0:
                        giniLR = (Y_LR.count(1) * Y_LR.count(0)) /len(Y_LR)
                    gini = giniLL + giniLR + giniRL + giniRR
                    result = 'LL :',Y_LL ,'LR:',Y_LR, 'RL :',Y_RL ,'RR:',Y_RR
                    result_list.append(result)
                    print('一階左右都有:',result)
                    gini_list.append(gini)
min_gini=np.where(gini_list==np.min(gini_list))
for i in min_gini[0]:
    print(i)
    print('best split:',result_list[i])


一階左空: ('LL:', [], 'LR:', [], 'RL :', [], 'RR', [0.0, 1.0, 1.0, 1.0, 0.0])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [0.0], 'RR', [1.0, 1.0, 1.0, 0.0])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [0.0, 1.0], 'RR', [1.0, 1.0, 0.0])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [0.0, 1.0, 1.0], 'RR', [1.0, 0.0])
一階左空: ('LL:', [], 'LR:', [], 'RL :', [0.0, 1.0, 1.0, 1.0], 'RR', [0.0])
一階左右都有: ('LL :', [], 'LR:', [0.0], 'RL :', [], 'RR:', [1.0, 1.0, 1.0, 0.0])
一階左右都有: ('LL :', [], 'LR:', [0.0], 'RL :', [1.0], 'RR:', [1.0, 1.0, 0.0])
一階左右都有: ('LL :', [], 'LR:', [0.0], 'RL :', [1.0, 1.0], 'RR:', [1.0, 0.0])
一階左右都有: ('LL :', [], 'LR:', [0.0], 'RL :', [1.0, 1.0, 1.0], 'RR:', [0.0])
一階左右都有: ('LL :', [], 'LR:', [0.0, 1.0], 'RL :', [], 'RR:', [1.0, 1.0, 0.0])
一階左右都有: ('LL :', [], 'LR:', [0.0, 1.0], 'RL :', [1.0], 'RR:', [1.0, 0.0])
一階左右都有: ('LL :', [], 'LR:', [0.0, 1.0], 'RL :', [1.0, 1.0], 'RR:', [0.0])
一階左右都有: ('LL :', [0.0], 'LR:', [1.0], 'RL :', [], 'RR:', [1.0, 1.0, 0.0])
一階左右都有: ('LL :', [0.0], 'LR:', [1.0], 'RL 

In [53]:
##depth=0
gini_list = []
gini = 0

for k in range(len(df['Y'])):
    gini += (df.iloc[k,1] - (sum(df['Y'])/len(df['Y'])))**2
result = 'root node:',(df['Y'].values)
print('depth=0:',result)
gini_list.append(gini) ##weighted mean square error
best_split =result


for j in cutpoint:
    giniR, giniL = 0, 0
    ##depth = 1
    X_L = get_XL(j, df)
    X_R = get_XR(j, df)
    Y_L = get_YL(j, df)
    Y_R = get_YR(j, df)

    if len(Y_R)>0:
        giniR = (Y_R.count(1) * Y_R.count(0)) /len(Y_R) 
    if len(Y_L)>0:
        giniL = (Y_L.count(1) * Y_L.count(0)) /len(Y_L)

    gini = giniR + giniL

    result = 'left :',Y_L ,'right:',Y_R
    print('depth=1:',result)
    if gini < min(gini_list):
        best_split = result
    gini_list.append(gini)

print('best split:',best_split)

depth=0: ('root node:', array([0., 1., 1., 1., 0.]))
depth=1: ('left :', [], 'right:', [0.0, 1.0, 1.0, 1.0, 0.0])
depth=1: ('left :', [0.0], 'right:', [1.0, 1.0, 1.0, 0.0])
depth=1: ('left :', [0.0, 1.0], 'right:', [1.0, 1.0, 0.0])
depth=1: ('left :', [0.0, 1.0, 1.0], 'right:', [1.0, 0.0])
depth=1: ('left :', [0.0, 1.0, 1.0, 1.0], 'right:', [0.0])
best split: ('left :', [0.0], 'right:', [1.0, 1.0, 1.0, 0.0])


In [54]:

# any number in (0.01, 0.32] is ok
the_cut = 0.09
X_L, X_R, Y_L, Y_R = get_XL(the_cut, df), get_XR(the_cut, df), get_YL(the_cut, df),get_YR(the_cut, df)
for m in X_L:
    for n in X_R:
        if m >= n:
            pass
        else:
            df_L = pd.DataFrame({'X':X_L, 'Y':Y_L})
            X_LL, X_LR, Y_LL, Y_LR = get_XL(m, df_L), get_XR(m, df_L), get_YL(m, df_L), get_YR(m, df_L)
            
            df_R = pd.DataFrame({'X':X_R, 'Y':Y_R})
            X_RL, X_RR, Y_RL, Y_RR = get_XL(n, df_R), get_XR(n, df_R), get_YL(n, df_R), get_YR(n, df_R)

            if len(Y_RL)>0:
                giniRL = (Y_RL.count(1) * Y_RL.count(0)) /len(Y_RL) 
            if len(Y_RR)>0:
                giniRR = (Y_RR.count(1) * Y_RR.count(0)) /len(Y_RR)
            if len(Y_LL)>0:
                giniLL = (Y_LL.count(1) * Y_LL.count(0)) /len(Y_LL)
            if len(Y_LR)>0:
                giniLR = (Y_LR.count(1) * Y_LR.count(0)) /len(Y_LR)
            gini = giniLL + giniLR + giniRL + giniRR
            result = 'LL :',Y_LL ,'LR:',Y_LR, 'RL :',Y_RL ,'RR:',Y_RR
            result_list.append(result)
            # print('一階左右都有:',result)
            gini_list.append(gini)
min_gini=np.where(gini_list==np.min(gini_list))
for i in min_gini[0]:
    print('best split:',result_list[i])


best split: ('LL :', [], 'LR:', [0.0, 1.0], 'RL :', [], 'RR:', [1.0, 1.0, 0.0])


In [24]:
the_cut = 0.18
result_list = []
gini_list = []
X_L, X_R, Y_L, Y_R = get_XL(the_cut, df), get_XR(the_cut, df), get_YL(the_cut, df),get_YR(the_cut, df)
for m in X_L:
    for n in X_R:
        if m >= n:
            pass
        else:
            df_L = pd.DataFrame({'X':X_L, 'Y':Y_L})
            X_LL, X_LR, Y_LL, Y_LR = get_XL(m, df_L), get_XR(m, df_L), get_YL(m, df_L), get_YR(m, df_L)
            
            df_R = pd.DataFrame({'X':X_R, 'Y':Y_R})
            X_RL, X_RR, Y_RL, Y_RR = get_XL(n, df_R), get_XR(n, df_R), get_YL(n, df_R), get_YR(n, df_R)

            if len(Y_RL)>0:
                giniRL = (Y_RL.count(1) * Y_RL.count(0)) /len(Y_RL) 
            if len(Y_RR)>0:
                giniRR = (Y_RR.count(1) * Y_RR.count(0)) /len(Y_RR)
            if len(Y_LL)>0:
                giniLL = (Y_LL.count(1) * Y_LL.count(0)) /len(Y_LL)
            if len(Y_LR)>0:
                giniLR = (Y_LR.count(1) * Y_LR.count(0)) /len(Y_LR)
            gini = giniLL + giniLR + giniRL + giniRR
            result = 'LL :',Y_LL ,'LR:',Y_LR, 'RL :',Y_RL ,'RR:',Y_RR
            result_list.append(result)
            # print('一階左右都有:',result)
            gini_list.append(gini)
min_gini=np.where(gini_list==np.min(gini_list))
print(min_gini)
print(len(result_list))
for i in min_gini[0]:
    print('best split:',result_list[i])

(array([2]),)
4
best split: ('LL :', [], 'LR:', [0.0], 'RL :', [1.0, 1.0, 1.0], 'RR:', [0.0])
